In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from arch import arch_model
from statsmodels.tsa.statespace.structural import UnobservedComponents
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, ParameterGrid
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
# ARIMA
def arima_grid_search(price_data, p_values, d_values, q_values):
    tscv = TimeSeriesSplit(n_splits=3)
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                try:
                    for train_index, test_index in tscv.split(price_data):
                        train, test = price_data[train_index], price_data[test_index]
                        model = ARIMA(train, order=(p,d,q))
                        model_fit = model.fit()
                        predictions = model_fit.forecast(steps=len(test))
                        error = mean_squared_error(test, predictions)
                        if error < best_score:
                            best_score, best_cfg = error, (p,d,q)
                except:
                    continue
    return best_cfg

# Load your time series data
data = pd.read_csv('time_series.csv', index_col='Date', parse_dates=True)
price_data = data['Price']

# Define range of parameters
p_values = range(0, 3)
d_values = range(0, 2)
q_values = range(0, 3)

# Perform grid search
best_order = arima_grid_search(price_data.values, p_values, d_values, q_values)
print(f'Best ARIMA order: {best_order}')

# Fit the final ARIMA model using the best parameters
final_model = ARIMA(price_data, order=best_order)
final_arima_model = final_model.fit()

# Forecast the next 10 steps
forecast = final_arima_model.forecast(steps=10)

In [ ]:
# SARIMA
def sarima_grid_search(price_data, p_values, d_values, q_values, P_values, D_values, Q_values, m_values):
    tscv = TimeSeriesSplit(n_splits=2)
    best_score, best_cfg = float("inf"), None
    for t in trends:
        for p in p_values:
            for d in d_values:
                for q in q_values:
                    for P in P_values:
                        for D in D_values:
                            for Q in Q_values:
                                for m in m_values:
                                    try:
                                        for train_index, test_index in tscv.split(price_data):
                                            train, test = price_data[train_index], price_data[test_index]
                                            model = SARIMAX(train, order=(p,d,q), seasonal_order=(P,D,Q,m), trend = t)
                                            model_fit = model.fit(disp=False)
                                            predictions = model_fit.forecast(steps=len(test))
                                            error = mean_squared_error(test, predictions)
                                            if error < best_score:
                                                best_score, best_cfg = error, (p,d,q,P,D,Q,m,t)
                                    except:
                                        continue
    return best_cfg

# Define range of parameters
p_values = range(0, 2)
d_values = range(0, 2)
q_values = range(0, 2)
P_values = range(0, 2)
D_values = range(0, 2)
Q_values = range(0, 2)
m_values = [30, 90, 365] # Adjust depending on your data
trends = ['n', 'c', 't', 'ct']

# Perform grid search
best_cfg = sarima_grid_search(price_data.values, p_values, d_values, q_values, P_values, D_values, Q_values, m_values)
print(f'Best SARIMA order: {best_cfg}')

# Fit the final SARIMA model using the best parameters
final_model = SARIMAX(price_data, order=best_cfg[:3], seasonal_order=best_cfg[3:7], trend = best_cfg[7])
final_sarima_model = final_model.fit()

# Forecast the next 10 steps
forecast = final_sarima_model.get_forecast(steps=10).predicted_mean

In [ ]:
#  Holt-Winters Exponential Smoothing 
def holtwinters_grid_search(price_data, trend_options, seasonal_options, seasonal_periods):
    tscv = TimeSeriesSplit(n_splits=3)
    best_score, best_cfg = float("inf"), None
    param_grid = {'trend': trend_options, 'seasonal': seasonal_options, 'seasonal_periods': [seasonal_periods]}
    for params in ParameterGrid(param_grid):
        try:
            for train_index, test_index in tscv.split(price_data):
                train, test = price_data[train_index], price_data[test_index]
                model = ExponentialSmoothing(train, **params)
                model_fit = model.fit()
                predictions = model_fit.forecast(steps=len(test))
                error = mean_squared_error(test, predictions)
                if error < best_score:
                    best_score, best_cfg = error, params
        except:
            continue
    return best_cfg

# Define range of parameters
trend_options = ['add', 'mul', None]
seasonal_options = ['add', 'mul', None]
seasonal_periods = [30, 90, 365]  # Adjust based on your data

# Perform grid search
best_params = holtwinters_grid_search(price_data.values, trend_options, seasonal_options, seasonal_periods)
print(f'Best Holt-Winters parameters: {best_params}')

# Fit the final Holt-Winters model using the best parameters
final_model = ExponentialSmoothing(price_data, **best_params)
final_hw_model = final_model.fit()

# Forecast the next 10 steps
forecast = final_hw_model.forecast(steps=10)



In [ ]:
# Define a function to find the best GARCH parameters using TimeSeriesSplit
def garch_grid_search(price_data, p_values, q_values):
    tscv = TimeSeriesSplit(n_splits=3)
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for q in q_values:
            try:
                for train_index, test_index in tscv.split(price_data):
                    train, test = price_data[train_index], price_data[test_index]
                    model = arch_model(train, vol='Garch', p=p, q=q)
                    model_fit = model.fit(disp="off")
                    predictions = model_fit.forecast(horizon=len(test)).mean.values[-1, :]
                    error = mean_squared_error(test, predictions)
                    if error < best_score:
                        best_score, best_cfg = error, (p, q)
            except:
                continue
    return best_cfg

# Define range of parameters
p_values = range(1, 3)
q_values = range(1, 3)

# Perform grid search
best_params = garch_grid_search(price_data.values, p_values, q_values)
print(f'Best GARCH parameters: {best_params}')

# Fit the final GARCH model using the best parameters
final_model = arch_model(price_data, vol='Garch', p=best_params[0], q=best_params[1])
final_garch_model = final_model.fit()

# Forecast the next 10 steps
forecast = final_garch_model.forecast(horizon=10)
forecasted_mean = forecast.mean.values[-1, :]

# Convert forecasted volatility back to prices
forecasted_prices = price_data[-1] + np.cumsum(forecasted_mean)

In [ ]:
# Define a function to find the best UCM parameters using TimeSeriesSplit
def ucm_grid_search(price_data, trend_options, seasonal_options):
    tscv = TimeSeriesSplit(n_splits=3)
    best_score, best_cfg = float("inf"), None
    param_grid = {'level': ['local level'], 'trend': trend_options, 'seasonal': seasonal_options}
    for params in ParameterGrid(param_grid):
        try:
            for train_index, test_index in tscv.split(price_data):
                train, test = price_data[train_index], price_data[test_index]
                model = UnobservedComponents(train, **params)
                model_fit = model.fit()
                predictions = model_fit.forecast(steps=len(test))
                error = mean_squared_error(test, predictions)
                if error < best_score:
                    best_score, best_cfg = error, params
        except:
            continue
    return best_cfg

# Define range of parameters
trend_options = [None, 'add', 'mul']
seasonal_options = [None, 30, 90, 365]  # Adjust based on your data

# Perform grid search
best_params = ucm_grid_search(price_data.values, trend_options, seasonal_options)
print(f'Best UCM parameters: {best_params}')

# Fit the final UCM model using the best parameters
final_model = UnobservedComponents(price_data, **best_params)
final_ucm_model = final_model.fit()

# Forecast the next 10 steps
forecast = final_ucm_model.get_forecast(steps=10).predicted_mean